In [7]:
# intialize df from csv
# select 100-200 at random (2048 tokens or less)
# use a list of questions and randomly assign them to each row
# answer questions using openAPI

import openai
import pandas as pd

In [ ]:
df = pd.read_csv('workout_data.csv')
df.head()

In [2]:
df['Time Per Workout'] = df['Time Per Workout'].fillna('')
df['Days Per Week'] = df['Days Per Week'].fillna('')
df['Recommended Supps'] = df['Recommended Supps'].fillna('')

In [3]:
from transformers import GPT2TokenizerFast

import numpy as np
from nltk.tokenize import sent_tokenize

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text):
    """count the number of tokens in a string"""
    return len(tokenizer.encode(text))

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
total_count = 0
token_count_array = []
for index, row in df.iterrows():
    for item in row:
        if isinstance(item, str):
            total_count += count_tokens(item)
    token_count_array.append(total_count)
    total_count = 0
df['Tokens'] = token_count_array
df.head()

Token indices sequence length is longer than the specified maximum sequence length for this model (1727 > 1024). Running this sequence through the model will result in indexing errors


,Main Goal,Workout Type,Training Level,Program Duration,Days Per Week,Time Per Workout,Equipment Required,Target Gender,Recommended Supps,Workout Description,Tokens
0,Lose Fat,Full Body,Beginner,12 weeks,4.0,30-45 minutes,"Barbell, Bodyweight, Cables, Dumbbells, Kettle...",Male & Female,Fat Burner Whey Protein BCAA Intra-Workout Fis...,This workout plan is designed to help you shre...,1783
1,Build Muscle,Split,Advanced,10 weeks,4.0,50 minutes,"Barbell, Bodyweight, Dumbbells, Machines",Male,Whey Protein Creatine Monohydrate Essential Fa...,This workout is designed to increase your musc...,432
2,Build Muscle,Split,Intermediate,12 weeks,6.0,45-60 minutes,"Barbell, Bodyweight, Dumbbells, Machines",Male & Female,Whey Protein Creatine Multivitamin BCAAs Citru...,Bulldozer Training inspired me to a created a ...,2311
3,Build Muscle,Split,Intermediate,12 weeks,5.0,45-60 minutes,"Bodyweight, Dumbbells",Male & Female,Whey Protein Protein Bars (Optional) Multivita...,So youve completed the 3 day full body dumbbel...,702
4,Build Muscle,Split,Intermediate,8 weeks,4.0,,"Barbell, Bodyweight, Cables, Dumbbells, Machines",Male & Female,Protein Multivitamins BCAAs,Summer is in the past and people are now looki...,1528


In [5]:
def select_random_rows(df, num_rows):
    # Select rows where the specified column is equal to the specified value
    filtered_df = df[df['Tokens'] <= 2048]
    
    selected_rows = filtered_df.sample(n=num_rows)
    
    return selected_rows

In [6]:
random_rows = select_random_rows(df, 200)
new_df = pd.DataFrame(random_rows)

In [7]:
new_df['context'] = new_df["Main Goal"] + "\n" + new_df["Workout Type"] + "\n" + new_df["Training Level"] + "\n" + new_df["Training Level"] + "\n" + new_df["Program Duration"] + "\n" + new_df["Days Per Week"].astype(str) + "\n" + new_df["Time Per Workout"] + "\n" + new_df["Equipment Required"] + "\n" + new_df["Target Gender"] + "\n" + new_df["Recommended Supps"] + "\n\n" + new_df["Workout Description"]
new_df.head()

,Main Goal,Workout Type,Training Level,Program Duration,Days Per Week,Time Per Workout,Equipment Required,Target Gender,Recommended Supps,Workout Description,Tokens,context
1083,Build Muscle,Full Body,Beginner,6 weeks,1.0,30-45 minutes,Bodyweight,Male & Female,,Go through each of these exercises 3 times tak...,169,Build Muscle\nFull Body\nBeginner\nBeginner\n6...
366,Build Muscle,Full Body,Beginner,20 weeks,3.0,30-45 minutes,"Barbell, Bodyweight, Cables, Dumbbells, EZ Bar",Male & Female,Weight Gain Shake Creatine Monohydrate Pre-Wor...,If youre just starting out the gym can be over...,1111,Build Muscle\nFull Body\nBeginner\nBeginner\n2...
3191,Increase Endurance,Full Body,Beginner,6 weeks,3.0,15-30 minutes,Bodyweight,Male & Female,Protein Powder (post-workout),Workout for a Cure Countdown to fitness 10090...,543,Increase Endurance\nFull Body\nBeginner\nBegin...
797,Build Muscle,Split,Intermediate,10 weeks,4.0,60-90 minutes,"Barbell, Bodyweight, Cables, Dumbbells",Female,Protein Powder Creatine Monohydrate Women's Mu...,Women can truly benefit from the increased tra...,1345,Build Muscle\nSplit\nIntermediate\nIntermediat...
1704,Increase Strength,Full Body,Beginner,12 weeks,3.0,90 minutes,"Barbell, Cables, EZ Bar",Male & Female,Whey Protein Creatine Fish Oil,For more information on Jason Blaha and Ice Cr...,1083,Increase Strength\nFull Body\nBeginner\nBeginn...


In [8]:
muscle_building_questions_list = [
"How can I build muscle effectively and efficiently?",
"What type of resistance training is best for building muscle?",
"How many reps and sets should I do to build muscle?",
"How much weight should I be lifting to see results?",
"What is the best way to target specific muscle groups?",
"How often should I workout to build muscle?",
"Are there any specific exercises that are best for building muscle?",
"How important is diet in building muscle?",
"How do I incorporate rest and recovery into my muscle building routine?",
"Can you help me create a balanced workout routine that includes cardio and strength training for building muscle?",
"What supplements do you recommend for my fitness goals?",
"How do supplements interact with any medications I may be taking?",
"Are there any supplements that are specifically for post-workout recovery?",
"Can you provide guidance on which supplements to use for specific fitness goals or needs?",
"What are my fitness goals and how can you help me achieve them?",
"How often should I workout and for how long?",
"What types of exercises do you recommend for my specific goals?",
"Can you help me create a balanced workout routine that includes cardio and strength training?",
"How do I incorporate rest and recovery into my routine?"
]

weight_loss_questions_list =[
"What is the best way to lose weight effectively and sustainably?",
"How can I incorporate both diet and exercise into my weight loss plan?",
"What types of exercises are best for burning fat and losing weight?",
"How many calories should I be consuming to lose weight?",
"How can I incorporate resistance training into my weight loss routine?",
"How do I track my progress and measure success?",
"How important is rest and recovery in a weight loss plan?",
"Are there any specific supplements that can aid in weight loss?",
"Can you help me create a comprehensive weight loss plan that includes both short-term and long-term goals?",
"What supplements do you recommend for my fitness goals?",
"How do supplements interact with any medications I may be taking?",
"Are there any supplements that are specifically for post-workout recovery?",
"Can you provide guidance on which supplements to use for specific fitness goals or needs?",
"What are my fitness goals and how can you help me achieve them?",
"How often should I workout and for how long?",
"What types of exercises do you recommend for my specific goals?",
"Can you help me create a balanced workout routine that includes cardio and strength training?",
"How do I incorporate rest and recovery into my routine?"
]

In [16]:
import numpy as np

def add_random_questions(df):
    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        # Select a random item from the list
        if row['Main Goal'] == "Lose Fat":
            random_question = np.random.choice(weight_loss_questions_list)
        else:
            random_question = np.random.choice(muscle_building_questions_list)
        
        # Add the selected item to the specified column in the current row
        df.at[index, 'questions'] = random_question
    
    return df

In [17]:
new_df = add_random_questions(new_df)
new_df['questions'] = "1." + new_df.questions
new_df.head()

,Main Goal,Workout Type,Training Level,Program Duration,Days Per Week,Time Per Workout,Equipment Required,Target Gender,Recommended Supps,Workout Description,Tokens,context,questions
1083,Build Muscle,Full Body,Beginner,6 weeks,1.0,30-45 minutes,Bodyweight,Male & Female,,Go through each of these exercises 3 times tak...,169,Build Muscle\nFull Body\nBeginner\nBeginner\n6...,1.How important is diet in building muscle?
366,Build Muscle,Full Body,Beginner,20 weeks,3.0,30-45 minutes,"Barbell, Bodyweight, Cables, Dumbbells, EZ Bar",Male & Female,Weight Gain Shake Creatine Monohydrate Pre-Wor...,If youre just starting out the gym can be over...,1111,Build Muscle\nFull Body\nBeginner\nBeginner\n2...,1.Can you help me create a balanced workout ro...
3191,Increase Endurance,Full Body,Beginner,6 weeks,3.0,15-30 minutes,Bodyweight,Male & Female,Protein Powder (post-workout),Workout for a Cure Countdown to fitness 10090...,543,Increase Endurance\nFull Body\nBeginner\nBegin...,1.Can you help me create a balanced workout ro...
797,Build Muscle,Split,Intermediate,10 weeks,4.0,60-90 minutes,"Barbell, Bodyweight, Cables, Dumbbells",Female,Protein Powder Creatine Monohydrate Women's Mu...,Women can truly benefit from the increased tra...,1345,Build Muscle\nSplit\nIntermediate\nIntermediat...,1.How many reps and sets should I do to build ...
1704,Increase Strength,Full Body,Beginner,12 weeks,3.0,90 minutes,"Barbell, Cables, EZ Bar",Male & Female,Whey Protein Creatine Fish Oil,For more information on Jason Blaha and Ice Cr...,1083,Increase Strength\nFull Body\nBeginner\nBeginn...,1.Can you help me create a balanced workout ro...


In [18]:
def get_answers(row):
    try:
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=f"Write questions based on the text below\n\nText: {row.context}\n\nQuestions:\n{row.questions}\n\nAnswers:\n1.",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response['choices'][0]['text']
    except Exception as e:
        print (e)
        return ""

In [20]:
new_df['answers']= new_df.apply(get_answers, axis=1)
new_df['answers'] = "1." + new_df.answers
new_df = new_df.dropna().reset_index().drop('index',axis=1)
print(new_df[['answers']].values[0][0])

1.Diet is an important factor in building muscle. Eating a balanced diet with adequate amounts of protein, carbohydrates, and healthy fats is essential for muscle growth.


In [21]:
new_df.to_csv("workout_data_qa_small.csv", index=False)

In [22]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(new_df, test_size=0.2, random_state=42)
len(train_df), len(test_df)

(160, 40)

In [23]:
import random

def create_fine_tuning_dataset(df, discriminator=False, n_negative=1, add_related=False):
    """
    Create a dataset for fine tuning the OpenAI model; either for a discriminator model, 
    or a model specializing in Q&A, where it says if no relevant context is found.

    Parameters
    ----------
    df: pd.DataFrame
        The dataframe containing the question, answer and context pairs
    discriminator: bool
        Whether to create a dataset for the discriminator
    n_negative: int
        The number of random negative samples to add (using a random context)
    add_related: bool
        Whether to add the related contexts to the correct context. These are hard negative examples

    Returns
    -------
    pd.DataFrame
        The dataframe containing the prompts and completions, ready for fine-tuning
    """
    rows = []
    for i, row in df.iterrows():
        for q, a in zip(("1." + row.questions).split('\n'), ("1." + row.answers).split('\n')):
            if len(q) >10 and len(a) >10:
                if discriminator:
                    rows.append({"prompt":f"{row.context}\nQuestion: {q[2:].strip()}\n Related:", "completion":f" yes"})
                else:
                    rows.append({"prompt":f"{row.context}\nQuestion: {q[2:].strip()}\nAnswer:", "completion":f" {a[2:].strip()}"})

    return pd.DataFrame(rows) 

In [24]:
for train_test, dt in [('train', train_df), ('test', test_df)]:
    ft = create_fine_tuning_dataset(dt, discriminator=False, n_negative=1, add_related=False)
    ft.to_json(f'qa_{train_test}.jsonl', orient='records', lines=True)

In [25]:
!openai tools fine_tunes.prepare_data -f qa_train.jsonl -q

Analyzing...

- Your file contains 160 prompt-completion pairs
- There are 3 duplicated prompt-completion sets. These are rows: [19, 134, 159]
- There are 2 examples that are very long. These are rows: [114, 157]
For conditional generation, and for classification the examples shouldn't be longer than 2048 tokens.
- All prompts end with suffix `?\nAnswer:`
- All completions end with suffix `.`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 3 duplicate rows [Y/n]: Y
- [Recommended] Remove 2 long examples [Y/n]: Y
The indices of the long examples has changed as a result of a previously applied recommendation.
The 2 long examples to be dropped are now at the following indices: [113, 155]


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `qa_train_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "qa_train_prepared.jsonl"

After you’ve fine-tuned a mod

In [26]:
!openai tools fine_tunes.prepare_data -f qa_test.jsonl -q

Analyzing...

- Your file contains 40 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `?\nAnswer:`
- All completions end with suffix `.`

No remediations found.

You can use your file for fine-tuning:
> openai api fine_tunes.create -t "qa_test.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `?\nAnswer:` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=["."]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 2.99 minutes to train a `curie` model, and less for `ada` and `babbage`. Queue will approximately take half an hour per job ahead of you.


In [8]:
training_file = openai.File.create(
  file=open("qa_train_prepared.jsonl"),
  purpose='fine-tune'
)
training_file_id = training_file['id']

In [9]:
validation_file = openai.File.create(
  file=open("qa_test.jsonl"),
  purpose='fine-tune'
)
validation_file_id = validation_file['id']

In [10]:
training_file_id

'file-qYyWKXUhnAF8AJExwasFsAQ0'

In [11]:
validation_file_id

'file-asRsFHYmhwVhVbjBrB7MIgJe'

In [12]:
fine_tune_job = openai.FineTune.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="davinci")


In [13]:
fine_tune_job

<FineTune fine-tune id=ft-c3Ka2aefJLpMBUsGbwHYfjAk at 0x16b1f00ddb0> JSON: {
  "created_at": 1675619544,
  "events": [
    {
      "created_at": 1675619545,
      "level": "info",
      "message": "Created fine-tune: ft-c3Ka2aefJLpMBUsGbwHYfjAk",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-c3Ka2aefJLpMBUsGbwHYfjAk",
  "model": "davinci",
  "object": "fine-tune",
  "organization_id": "org-6dS2QHofwzzk85aaYOlnS3cr",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 812543,
      "created_at": 1675619515,
      "filename": "file",
      "id": "file-qYyWKXUhnAF8AJExwasFsAQ0",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1675619545,
  "validation_files": [
    {
      "bytes": 228361,
  

In [15]:
openai.FineTune.list()

<OpenAIObject list at 0x16b1f01cc20> JSON: {
  "data": [
    {
      "created_at": 1675619544,
      "fine_tuned_model": "davinci:ft-personal-2023-02-05-18-06-13",
      "hyperparams": {
        "batch_size": 1,
        "learning_rate_multiplier": 0.1,
        "n_epochs": 4,
        "prompt_loss_weight": 0.01
      },
      "id": "ft-c3Ka2aefJLpMBUsGbwHYfjAk",
      "model": "davinci",
      "object": "fine-tune",
      "organization_id": "org-6dS2QHofwzzk85aaYOlnS3cr",
      "result_files": [
        {
          "bytes": 37131,
          "created_at": 1675620375,
          "filename": "compiled_results.csv",
          "id": "file-AutFbQNDR4RdzggzSchj1xW0",
          "object": "file",
          "purpose": "fine-tune-results",
          "status": "processed",
          "status_details": null
        }
      ],
      "status": "succeeded",
      "training_files": [
        {
          "bytes": 812543,
          "created_at": 1675619515,
          "filename": "file",
          "id": "file

In [19]:
def answer_question(question):
    openai.Completion.create(
        model="davinci:ft-personal-2023-02-05-18-06-13",
        prompt= prompt = f"Question: {question}\n Related:"
    )

In [20]:
results = answer_question("Create me a workout plan that focuses on muscle building")